<font color="magenta" size=7>Source Code

This code replicates the braingeneers [deepspike dashboad](http://braingeneers.gi.ucsc.edu/dashboard/deepspike). The source code that david parks wrote can be [found here](https://github.com/braingeneers/dashboard/blob/master/apps/app_deepspike.py). 

## <font color="orange"> Errors to check

<font color="red">There was an error when:</font>
``` python
data_files_basepath='s3://braingeneers/personal/dfparks/deepspike/dataset_zenodo/'
```
**fix this!**
```
NotFoundError: Object s3://braingeneers/personal/dfparks/deepspike/dataset_zenodo/recordings_36cells_four-tetrodes_30.0_10.0uV_20-06-2019_14_48.h5 does not exist
```

# Set Up Notebook

Import packages

In [ ]:
import ipywidgets as ipw
import boto3 # look up what this package is
import os
import tensorflow.compat.v1 as tf
import h5py

Download Main dataset locally

In [ ]:
!aws --endpoint https://s3.nautilus.optiputer.net s3 cp s3://braingeneers/personal/dfparks/deepspike/dataset_zenodo/recordings_36cells_four-tetrodes_30.0_10.0uV_20-06-2019_14_48.h5  .


Download call results locally

In [ ]:
!aws --endpoint https://s3.nautilus.optiputer.net s3 cp s3://braingeneers/personal/dfparks/deepspike/results/spikesortv2/zenodo_results.call.json .


# <font color="gray">Helper Class: NeuralDataReaderZenodo

In [ ]:
# Raw data reader for Zenodo data. Will be replaced when I move to a more generic dataset format.
class NeuralDataReaderZenodo:
    FILE_CACHE = {}

    def __init__(self, data_files: (tuple, list), data_files_basepath: str):
        self.data_files = data_files
        self.data_files_basepath = data_files_basepath
        self._data_files_sample_counts = None  # lazy evaluation
        self._data_files_sample_counts_cumsum = None  # lazy evaluation

    """ Reads neural data from the Zenodo HDF5 file format. """
    def read_data(self, global_neural_offset: int, read_length: int, channels: (list, tuple) = None):
        data_file_index, local_neural_offset = self.resolve_local_neural_offset(global_neural_offset)
        data_file = self.data_files[data_file_index]
        if data_file_index not in NeuralDataReaderZenodo.FILE_CACHE:
            NeuralDataReaderZenodo.FILE_CACHE[data_file_index] = \
                tf.io.gfile.GFile(os.path.join(self.data_files_basepath, data_file), mode='rb')

        f = NeuralDataReaderZenodo.FILE_CACHE[data_file_index]
        h5f = h5py.File(f, 'r')

        channels_ixs = np.arange(h5f['recordings'].shape[0]) if channels is None else channels
        data = h5f['recordings'][channels_ixs, local_neural_offset:local_neural_offset + read_length]

        return data.T.astype(np.float32)

    def data_files_sample_counts(self):
        if self._data_files_sample_counts is None:
            self._data_files_sample_counts = []
            for df in self.data_files:
                with tf.io.gfile.GFile(os.path.join(self.data_files_basepath, df), mode='rb') as f:
                    h5f = h5py.File(f, 'r')
                    self._data_files_sample_counts.append(h5f['recordings'].shape[1])

        return self._data_files_sample_counts

    def resolve_local_neural_offset(self, global_neural_offset):
        """ Resolves a global_neural_offset to a data_files index and local neural_offset into that file """
        if self._data_files_sample_counts_cumsum is None:
            self._data_files_sample_counts_cumsum = np.cumsum(self.data_files_sample_counts())

        ix = np.searchsorted(self._data_files_sample_counts_cumsum, global_neural_offset, side='right')
        assert 0 <= ix < len(self.data_files)
        lno = global_neural_offset - self._data_files_sample_counts_cumsum[ix - 1] if ix > 0 else global_neural_offset

        return ix, lno


# <font color="gray">Helper Function: compute_call_spikes

In [ ]:
def compute_call_spikes(time_offset, channel_number, data_calls):
    called_spikes = [
        c for c in data_calls
        if time_offset - GRAPH_WIDTH < c['spike_center_ix'] < time_offset + GRAPH_WIDTH \
        and c['channel'] == channel_number
    ]
    return called_spikes

# Create `called_spikes`

I am using david's code to create the `called_spikes` variable that is fed into the <font color="blue">update_lfp_graph</font> function

In [ ]:
with open("zenodo_results.call.json") as f:
    data_calls = json.load(f)
called_spikes = compute_call_spikes(time_offset, channel_number, data_calls)

In [ ]:
#import apps.utils as utils

In [ ]:
#data_calls = json.load("zenodo_results.call.json")
#data_calls = json.loads( utils.read_s3_bytes(
#        bucket='braingeneers', key='personal/dfparks/deepspike/results/spikesortv2/zenodo_results.call.json'
#    ).decode('utf-8'))

In [ ]:
#data_calls = json.load("zenodo_results.call.json")

# update_lfp_graph <small>(line 446)

##  <font color="green"> widgets

### <font color="green"> Channel

In [ ]:
w_channel = ipw.IntSlider(value=10, min=0,max=30,description='Channel')

In [ ]:
w_channel

### <font color="green">Time Offset

In [ ]:
w_time_offset = ipw.IntSlider(value=1, min=0,max=10,description='Time Offset')

In [ ]:
w_time_offset

## <font color="blue">Davids Code

<font color="red">There was an error when:</font>
``` python
data_files_basepath='s3://braingeneers/personal/dfparks/deepspike/dataset_zenodo/'
```
**fix this!**
```
NotFoundError: Object s3://braingeneers/personal/dfparks/deepspike/dataset_zenodo/recordings_36cells_four-tetrodes_30.0_10.0uV_20-06-2019_14_48.h5 does not exist
```

Value from widget

In [ ]:
channel_number_int = w_channel.value

In [ ]:
time_offset = w_time_offset.value

Beginning variables to set: (line 33)

In [ ]:
WINDOW_SIZE = 500  # this needs to be somewhere standardized as well, necessary for converting [-1, +1] back
GRAPH_WIDTH = 1000
KERNEL_SIZE = int(WINDOW_SIZE * 0.4) + 1  # size of the median filter kernel
s3client = boto3.client('s3', endpoint_url="https://s3.nautilus.optiputer.net")

Set class for reading data

In [ ]:
reader = NeuralDataReaderZenodo(
    data_files=['recordings_36cells_four-tetrodes_30.0_10.0uV_20-06-2019_14_48.h5'],
    data_files_basepath=''
    #data_files_basepath='s3://braingeneers/personal/dfparks/deepspike/dataset_zenodo/',
    # data_files_basepath='/dashboard/tmp/app_deepspike/zenodo/'
)

In [ ]:
# Read the raw data, pad when the graph would extend beyond either side of the available data
global_neural_offset = max(0, time_offset - int(GRAPH_WIDTH//2))
pad_left = abs(min(0, time_offset - int(GRAPH_WIDTH//2)))
pad_right = abs(min(0, sum(reader.data_files_sample_counts()) - time_offset - GRAPH_WIDTH))

In [ ]:

read_length = GRAPH_WIDTH - pad_left + pad_right
data_local = reader.read_data(global_neural_offset=global_neural_offset,
                              read_length=read_length, channels=[channel_number_int])
data_local = np.squeeze(data_local)
data_local = np.pad(data_local, (pad_left, pad_right))

In [ ]:
assert len(data_local) == GRAPH_WIDTH
x_data = np.arange(time_offset - int(GRAPH_WIDTH//2), time_offset + int(GRAPH_WIDTH//2))

In [ ]:
waveforms = []
for c in called_spikes:
    w_npy = np.squeeze(waveform_distribution().generate_waveform(*c['parameters']))
    ix_start = int(c['spike_center_ix'] - (w_npy.shape[0] // 2))
    x = np.arange(ix_start, ix_start + w_npy.shape[0])
    waveforms.append((x, w_npy))

In [ ]:
figure = dict(
    data=[
        dict(
            x=x_data,
            y=data_local,
            name='Raw LFP Data',
            line=dict(color='rgb(55, 83, 109)', width=1),
        )
    ] + [
        dict(
            x=wfx,
            y=wfy,
            name='Predicted waveform',
            showlegend=False if count > 0 else True,  # enable only for the first
            line=dict(color='rgba(255, 0, 0, 0.65)', width=4)
        )
        for count, (wfx, wfy) in enumerate(waveforms)
    ],
    layout=dict(
        xaxis=dict(
            range=(time_offset - int(GRAPH_WIDTH//2), time_offset + int(GRAPH_WIDTH//2)),
        ),
        yaxis=dict(
            title='LFP',
        ),
        autosize=True,
        hovermode="closest",
        plot_bgcolor="#F9F9F9",
        paper_bgcolor="#F9F9F9",
        title='LFP Data',
        showlegend=True,
        legend=dict(x=0, y=1.0),
        shapes=[
            dict(
                type='rect',
                xref='x',
                yref='paper',
                x0=c['spike_center_ix'] - x, x1=c['spike_center_ix'] + x,
                y0=0, y1=1,
                fillcolor='Green',
                opacity=0.07,
                layer='above',
                line=dict(width=0),
            )
            for x in [50, 40, 30, 20, 10]
            for c in called_spikes
        ],
    ),
)